In [2]:
import sys
sys.path.insert(0, '/Users/matthewashman/github/MasterProject2018')

# Import necessary modules. Set settings. Import data.
import math
import numpy as np
import pandas as pd
import random
import pywt
import parser
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from statsmodels.robust import mad
from tsfresh.feature_extraction import feature_calculators
from FeatureExtraction.feature_tools import detect_peaks, get_s1s2
from IPython.display import display, clear_output, HTML

import pdb

plt.style.use('default')

file_loc = '/Users/matthewashman/github/MasterProject2018/Data/EPdata/'
with open('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/final_segment_labels.csv', 'r') as file_in:
    labels = pd.read_csv(file_in)

In [5]:
af_patients = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14') # AF patients
at_patients = ('1', '2', '3') # AT patients
avnrt_patients = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23') # AVNRT patients
avrt_patients = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13') # AVRT patients
ep_patients = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10') # EP patients
coupling_intervals = ('220','230', '240', '250', '260', '270', '280', '290', '300',
                      '310', '320', '330', '340', '350', '360', '370', '380', '390', '400')

X_af = epdata_main(file_loc, af_patients, 'af', tuple(reversed(coupling_intervals)))
X_at = epdata_main(file_loc, at_patients, 'at', list(reversed(coupling_intervals)))
X_avrt = epdata_main(file_loc, avrt_patients, 'avrt', list(reversed(coupling_intervals)))
X_avnrt = epdata_main(file_loc, avnrt_patients, 'avnrt', list(reversed(coupling_intervals)))
X_ep = epdata_main(file_loc, ep_patients, 'ep', list(reversed(coupling_intervals)))

X = pd.concat([X_af, X_at, X_avrt, X_avnrt, X_ep], ignore_index=True)

Parsing error: coupling interval 400ms not available for patient 1.
Parsing error: coupling interval 390ms not available for patient 1.
Parsing error: coupling interval 380ms not available for patient 1.
Parsing error: coupling interval 370ms not available for patient 1.
Parsing error: coupling interval 360ms not available for patient 1.
Parsing error: coupling interval 350ms not available for patient 1.
Parsing error: coupling interval 220ms not available for patient 1.
Parsing error: coupling interval 240ms not available for patient 2.
Parsing error: coupling interval 230ms not available for patient 2.
Parsing error: coupling interval 220ms not available for patient 2.
Parsing error: coupling interval 250ms not available for patient 3.
Parsing error: coupling interval 240ms not available for patient 3.
Parsing error: coupling interval 230ms not available for patient 3.
Parsing error: coupling interval 220ms not available for patient 3.
Parsing error: coupling interval 400ms not avail

Parsing error: non-integer values in channels V1/CS9-10 for coupling interval 280ms.
Parsing error: coupling interval 270ms not available for patient 1.
Parsing error: coupling interval 260ms not available for patient 1.
Parsing error: coupling interval 250ms not available for patient 1.
Parsing error: coupling interval 240ms not available for patient 1.
Parsing error: coupling interval 230ms not available for patient 1.
Parsing error: coupling interval 220ms not available for patient 1.
Parsing error: coupling interval 380ms not available for patient 2.
Parsing error: non-integer values in channels V1/CS9-10 for coupling interval 370ms.
Parsing error: coupling interval 360ms not available for patient 2.
Parsing error: non-integer values in channels V1/CS9-10 for coupling interval 350ms.
Parsing error: coupling interval 320ms not available for patient 2.
Parsing error: non-integer values in channels V1/CS9-10 for coupling interval 310ms.
Parsing error: coupling interval 300ms not avail

Parsing error: coupling interval 260ms not available for patient 1.
Parsing error: coupling interval 250ms not available for patient 1.
Parsing error: coupling interval 240ms not available for patient 1.
Parsing error: coupling interval 230ms not available for patient 1.
Parsing error: coupling interval 220ms not available for patient 1.
Parsing error: coupling interval 390ms not available for patient 2.
Parsing error: coupling interval 380ms not available for patient 2.
Parsing error: coupling interval 370ms not available for patient 2.
Parsing error: coupling interval 350ms not available for patient 2.
Parsing error: coupling interval 330ms not available for patient 2.
Parsing error: coupling interval 320ms not available for patient 2.
Parsing error: coupling interval 310ms not available for patient 2.
Parsing error: coupling interval 300ms not available for patient 2.
Parsing error: coupling interval 270ms not available for patient 2.
Parsing error: coupling interval 260ms not avail

Parsing error: coupling interval 350ms not available for patient 11.
Parsing error: coupling interval 330ms not available for patient 11.
Parsing error: coupling interval 310ms not available for patient 11.
Parsing error: coupling interval 260ms not available for patient 11.
Parsing error: coupling interval 250ms not available for patient 11.
Parsing error: coupling interval 240ms not available for patient 11.
Parsing error: coupling interval 230ms not available for patient 11.
Parsing error: coupling interval 220ms not available for patient 11.
Parsing error: coupling interval 390ms not available for patient 12.
Parsing error: coupling interval 380ms not available for patient 12.
Parsing error: coupling interval 370ms not available for patient 12.
Parsing error: coupling interval 360ms not available for patient 12.
Parsing error: coupling interval 350ms not available for patient 12.
Parsing error: coupling interval 340ms not available for patient 12.
Parsing error: coupling interval 3

Parsing error: coupling interval 310ms not available for patient 1.
Parsing error: coupling interval 270ms not available for patient 1.
Parsing error: coupling interval 260ms not available for patient 1.
Parsing error: coupling interval 250ms not available for patient 1.
Parsing error: coupling interval 240ms not available for patient 1.
Parsing error: coupling interval 230ms not available for patient 1.
Parsing error: coupling interval 220ms not available for patient 1.
Parsing error: coupling interval 400ms not available for patient 2.
Parsing error: non-integer values in channels V1/CS9-10 for coupling interval 390ms.
Parsing error: coupling interval 380ms not available for patient 2.
Parsing error: non-integer values in channels V1/CS9-10 for coupling interval 370ms.
Parsing error: non-integer values in channels V1/CS9-10 for coupling interval 360ms.
Parsing error: coupling interval 260ms not available for patient 2.
Parsing error: coupling interval 250ms not available for patient 

In [ ]:
%matplotlib qt
patient_types = X['Type'].unique()
# Get coupling intervals for consistent plotting (need scale to be the same for a fair comparison)
cis = np.sort(coupling_intervals)[::-1]

for patient_type in patient_types:
    patients = X[X['Type']==patient_type]['Patient'].unique()
    if patient_type == 'avrt':
        patients = patients[1:]
        
    for patient in patients:
        patient_name = patient_type.upper() + patient
        patient_X = X[(X['Type']==patient_type) & (X['Patient']==patient) & (X['S1/S2']=='S2')]
        
        patient_cis = patient_X['Coupling Interval'].unique()
        patient_cis = np.sort(patient_cis)[::-1]
        
        num_rows = len(cis) + 1
        
        fig = plt.figure(figsize=(16,9))
        
        # Need y limits to be constant - first find ymax/ymin
        cs12_ymax = 0
        cs12_ymin = 0
        cs34_ymax = 0
        cs34_ymin = 0
        cs56_ymax = 0
        cs56_ymin = 0
        for i,ci in enumerate(patient_cis):
            cs12 = patient_X[(patient_X['Coupling Interval']==ci) & (patient_X['Channel']=='CS1-2')].iloc[0]['Data']
            cs34 = patient_X[(patient_X['Coupling Interval']==ci) & (patient_X['Channel']=='CS3-4')].iloc[0]['Data']
            cs56 = patient_X[(patient_X['Coupling Interval']==ci) & (patient_X['Channel']=='CS5-6')].iloc[0]['Data']
            
            if max(cs12)>cs12_ymax:
                cs12_ymax = max(cs12)
            if min(cs12)<cs12_ymin:
                cs12_ymin = min(cs12)
                
            if max(cs34)>cs34_ymax:
                cs34_ymax = max(cs34)
            if min(cs34)<cs34_ymin:
                cs34_ymin = min(cs34)
                
            if max(cs56)>cs56_ymax:
                cs56_ymax = max(cs56)
            if min(cs56)<cs56_ymin:
                cs56_ymin = min(cs56)
                
        for i,ci in enumerate(cis):                   
            cs12_ax = plt.subplot(num_rows, 3, (3*(i+1)+1))
            cs34_ax = plt.subplot(num_rows, 3, (3*(i+1)+2))
            cs56_ax = plt.subplot(num_rows, 3, (3*(i+1)+3))
            
            # Plot if segment exists for given coupling interval
            if ci in patient_cis:
                # Get labels for displaying
                patient_labels = labels[(labels['Patient']==patient_name) & (labels['CI']==int(ci))]
                try:
                    cs12_label = patient_labels['CS1-2'].values[0]
                except KeyError:
                    cs12_label = 'error'  
                try:
                    cs34_label = patient_labels['CS3-4'].values[0]
                except:
                    cs34_label = 'error' 
                try:
                    cs56_label = patient_labels['CS5-6'].values[0]
                except:
                    cs56_label = 'error' 
                
                cs12 = patient_X[(patient_X['Coupling Interval']==ci) & (patient_X['Channel']=='CS1-2')].iloc[0]['Data']
                cs34 = patient_X[(patient_X['Coupling Interval']==ci) & (patient_X['Channel']=='CS3-4')].iloc[0]['Data']
                cs56 = patient_X[(patient_X['Coupling Interval']==ci) & (patient_X['Channel']=='CS5-6')].iloc[0]['Data']

                cs12_ax.plot(np.arange(len(cs12)), cs12, 'k-')
                cs12_ax.set_ylim([cs12_ymin,cs12_ymax])
                cs12_ax.set_ylabel((ci), fontweight='bold')
                cs12_ax.set_xticks([])
                cs12_ax.text(0.9, 0.15, cs12_label, horizontalalignment='center', 
                             verticalalignment='center', transform=cs12_ax.transAxes, fontweight='bold')

                cs34_ax.plot(np.arange(len(cs34)), cs34, 'k-')
                cs34_ax.set_ylim([cs34_ymin,cs34_ymax])
                cs34_ax.set_xticks([])
                cs34_ax.text(0.9, 0.15, cs34_label, horizontalalignment='center', 
                             verticalalignment='center', transform=cs34_ax.transAxes, fontweight='bold')

                cs56_ax.plot(np.arange(len(cs56)), cs56, 'k-')
                cs56_ax.set_ylim([cs56_ymin,cs56_ymax])
                cs56_ax.set_xticks([])
                cs56_ax.text(0.9, 0.15, cs56_label, horizontalalignment='center', 
                             verticalalignment='center', transform=cs56_ax.transAxes, fontweight='bold')
            
            # If not, plot empty axis
            else:
                cs12_ax = plt.subplot(num_rows, 3, (3*(i+1)+1))
                cs34_ax = plt.subplot(num_rows, 3, (3*(i+1)+2))
                cs56_ax = plt.subplot(num_rows, 3, (3*(i+1)+3))
                
                cs12_ax.set_ylim([cs12_ymin,cs12_ymax])
                cs12_ax.set_ylabel((ci), fontweight='bold')
                cs12_ax.set_xticks([])
                
                cs34_ax.set_ylim([cs34_ymin,cs34_ymax])
                cs34_ax.set_xticks([])
                
                cs56_ax.set_ylim([cs56_ymin,cs56_ymax])
                cs56_ax.set_xticks([])
                
                cs12_ax.text(0.5, 0.5, 'N/A', horizontalalignment='center', 
                             verticalalignment='center', transform=cs12_ax.transAxes)
                cs34_ax.text(0.5, 0.5, 'N/A', horizontalalignment='center', 
                             verticalalignment='center', transform=cs34_ax.transAxes)
                cs56_ax.text(0.5, 0.5, 'N/A', horizontalalignment='center', 
                             verticalalignment='center', transform=cs56_ax.transAxes)
                
            
        cs12_ax.set_xlabel('CS1-2', fontweight='bold')
        cs34_ax.set_xlabel('CS3-4', fontweight='bold')
        cs56_ax.set_xlabel('CS5-6', fontweight='bold')
        
        cs12_long = X[(X['Type']==patient_type) & (X['Patient']==patient) & (X['S1/S2']=='-1') & (X['Channel']=='CS1-2')]['Data'].values[0]
        cs34_long = X[(X['Type']==patient_type) & (X['Patient']==patient) & (X['S1/S2']=='-1') & (X['Channel']=='CS3-4')]['Data'].values[0]
        cs56_long = X[(X['Type']==patient_type) & (X['Patient']==patient) & (X['S1/S2']=='-1') & (X['Channel']=='CS5-6')]['Data'].values[0]
        
        red_patch = patches.Rectangle((45,min(cs12_long)),150,(max(cs12_long)-min(cs12_long)),facecolor='r', alpha=0.3, linewidth=1, edgecolor='k', linestyle='--')
        
        long_ax = plt.subplot(num_rows, 1, 1)
        long_ax.plot(np.arange(len(cs12_long)), cs12_long, 'k-')
        long_ax.add_patch(red_patch)
        long_ax.xaxis.tick_top()
        long_ax.set_xlabel('ms')
        long_ax.xaxis.set_label_position('top')
        
#         axes[0][0].plot(np.arange(len(cs12_long)), cs12_long, 'k-', range(50,200), cs12_long[50:200], 'r-')
#         axes[0][1].plot(np.arange(len(cs34_long)), cs34_long, 'k-', range(50,200), cs34_long[50:200], 'r-')
#         axes[0][2].plot(np.arange(len(cs56_long)), cs56_long, 'k-', range(50,200), cs56_long[50:200], 'r-')
        
#         axes[0][0].set_title('CS1-2')
#         axes[0][1].set_title('CS3-4')
#         axes[0][2].set_title('CS5-6')
    
        plt.title(patient_type + patient)
        plt.subplots_adjust(left=0.1, bottom=0.05, right=0.95, top=0.9, hspace=0.25)
#         plt.plot()
        plt.draw()
        plt.waitforbuttonpress()
#         plt.savefig('/Users/matthewashman/Documents/IIB/Project/Questionnaire_Images_New/' + patient_type + patient + '_s2_response_16x9.png')
        plt.close()
    

/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [3]:
def epdata_main(file_loc, patients, patient_type, coupling_intervals):
    X_list = [] # Initially store data in a list of dictionaries
    column_names = ['Type', 'Patient', 'Coupling Interval', 'S1/S2', 'Data']    # Column names of pandas dataframe
    for patient in patients:
        get_long_seg = True
        for coupling_interval in coupling_intervals:
            # Extract data from file
            # file_name = file_loc + 'TESTEXPORT' + file + '.txt'
            try:
                file_name = file_loc + patient_type + patient + '/' + patient_type + 'patient' + patient + '-0' + coupling_interval + '.txt'
                data, sr, num_samples = parser.parseFile(file_name)
            except FileNotFoundError:
                try:
                    file_name = file_loc + patient_type + patient + '/' + patient_type + 'patient0' + patient + '-0' + coupling_interval + '.txt'
                    data, sr, num_samples = parser.parseFile(file_name)
                except:
                    print('Parsing error: coupling interval ' + coupling_interval + 'ms not available for patient ' + patient + '.')
                    continue

            # Use the following channels
            try:
                cs34 = data['CS 3-4'].values
                cs56 = data['CS 5-6'].values
            except KeyError:
                try:
                    cs34 = data['CS3-4'].values
                    cs56 = data['CS5-6'].values
                except KeyError:
                    print('Parsing error: could not extract CS3-4 and CS5-6 values for coupling interval ' + coupling_interval + 'ms.')
                    continue
            try:
                s = data['CSp'].values
                cs12 = data['CSd'].values
            except KeyError:
                try:
                    s = data['CS9-10'].values
                    cs12 = data['CS1-2'].values
                except KeyError:
                    try:
                        s = data['CS 9-10'].values
                        cs12 = data['CS 1-2'].values
                    except KeyError:
                        print('Parsing error: could not extract channel values for coupling interval ' + coupling_interval + 'ms.')
                        continue

            channel_names = ('CS1-2', 'CS3-4', 'CS5-6')
            channels = (cs12, cs34, cs56)
            offsets = (15, 12, 12)
            if  all(isinstance(i, np.int64) for i in s):
                for channel, channel_name, offset in zip(channels, channel_names, offsets):
                    if all(isinstance(i, np.int64) for i in channel):
                        # Get segment
                        s1_segs, s2_seg = get_segment(channel, s, sr, int(coupling_interval), offset, False)
                        temp_dict = {}
                        temp_dict.update({'Type': patient_type, 'Patient': patient, 'Coupling Interval': coupling_interval, 'Channel': channel_name, 'S1/S2': 'S2', 'Data': s2_seg})
                        X_list.append(temp_dict)
                        for i,s1_seg in enumerate(s1_segs):
                            temp_dict = {}
                            temp_dict.update({'Type': patient_type, 'Patient': patient, 'Coupling Interval': coupling_interval, 'Channel': channel_name, 'S1/S2': 'S1', 'Data': s1_segs[i]})
                            X_list.append(temp_dict)
                            
                        if get_long_seg == True:
                            s2_seg = get_long_s2_segment(channel, s, sr, int(coupling_interval), offset)
                            temp_dict = {}
                            temp_dict.update({'Type': patient_type, 'Patient': patient, 'Coupling Interval': coupling_interval, 'Channel': channel_name, 'S1/S2': '-1', 'Data': s2_seg})
                            X_list.append(temp_dict)
                    else:
                        print('Parsing error: non-integer values in channel ' + channel_name + ' for coupling interval ' + coupling_interval + 'ms.')
                    
                get_long_seg = False
                
            else:
                print('Parsing error: non-integer values in channels V1/CS9-10 for coupling interval ' + coupling_interval + 'ms.')
                continue

    X = pd.DataFrame(X_list)
    return X

In [4]:
# Used to find S2 pulse using coupling_interval
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def get_segment(x, s, sr, coupling_interval=0, offset=10, plot_segment=True):
    # Get S1/S2 pulses from s
    s1s2_idxs = get_s1s2(s, sr, False)

    # Identify S2 pulse
    s1s2_intervals = np.diff(s1s2_idxs)
    s2_idx = find_nearest(s1s2_intervals, coupling_interval)
    s2_idx = s1s2_idxs[s2_idx+1]
    s1_idxs = np.asarray([i for i in s1s2_idxs if ((i>50) & (i<s2_idx))])
    # Add small offset
    s1_idxs += -5
    s2_idx += -5

    # For now just use strict segment length of 125.
    seg_length = 150
    s1_segs = []    # List of S1 segments
    s2_seg = x[s2_idx:(s2_idx+seg_length)] # S2 segment
    for s1_idx in s1_idxs:
        s1_segs.append(x[s1_idx:(s1_idx+seg_length)])


    if plot_segment:
        fig, [ax1, ax2] = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(16,4))
        ax1.plot(range(len(x)), x, range(s2_idx, s2_idx+seg_length), s2_seg, 'g')
        ax1.hold(True)
        for i, s1_seg in enumerate(s1_segs):
            ax1.plot(range(s1_idxs[i], (s1_idxs[i]+seg_length)), s1_seg, 'r')
        ax2.plot(range(len(s)), s)
        for idx in s1s2_idxs:
            ax2.axvline(idx, ymin=min(s), ymax=max(s), c='k', linestyle='--')
        plt.draw()
        plt.waitforbuttonpress()
        plt.close(fig)

    return np.array(s1_segs).astype(float), s2_seg

def get_long_s2_segment(x, s, sr, coupling_interval=0, offset=10):
    # Get S1/S2 pulses from s
    s1s2_idxs = get_s1s2(s, sr, False)

    # Identify S2 pulse
    s1s2_intervals = np.diff(s1s2_idxs)
    s2_idx = find_nearest(s1s2_intervals, coupling_interval)
    s2_idx = s1s2_idxs[s2_idx+1]
    # Add small offset
    s2_idx += - 50

    # For now just use strict segment length of 125.
    seg_length = 550
    s2_seg = x[s2_idx:(s2_idx+seg_length)] # S2 segment

    return s2_seg